# MONODEPTH on OpenVINO IR Model

This notebook demonstrates Monocular Depth Estimation with MidasNet in OpenVINO. Model information: https://github.com/openvinotoolkit/open_model_zoo/blob/master/models/public/midasnet/midasnet.md

<img src="monodepth.gif">

### What is Monodepth?
Monocular Depth Estimation is the task of estimating scene depth using a single image. It has many potential applications in robotics, 3D reconstruction, medical imaging and autonomous systems. For this demo, we use a neural network model called [MiDaS](https://github.com/intel-isl/MiDaS) which was developed by the Intelligent Systems Lab at Intel. Check out their research paper to learn more. 

R. Ranftl, K. Lasinger, D. Hafner, K. Schindler and V. Koltun, ["Towards Robust Monocular Depth Estimation: Mixing Datasets for Zero-shot Cross-dataset Transfer,"](https://ieeexplore.ieee.org/document/9178977) in IEEE Transactions on Pattern Analysis and Machine Intelligence, doi: 10.1109/TPAMI.2020.3019967.

## Preparation 

### Imports

In [ ]:
import os
import time
import urllib
from pathlib import Path

import cv2
import matplotlib.cm
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import (
    HTML,
    FileLink,
    Pretty,
    ProgressBar,
    Video,
    clear_output,
    display,
)
from openvino.inference_engine import IECore

### Settings

In [ ]:
DEVICE = "CPU"
MODEL_FILE = "models/MiDaS_small.xml"

model_name = os.path.basename(MODEL_FILE)
model_xml_path = Path(MODEL_FILE).with_suffix(".xml")

## Functions

In [ ]:
def normalize_minmax(data):
    """Normalizes the values in `data` between 0 and 1"""
    return (data - data.min()) / (data.max() - data.min())

In [ ]:
def load_image(path: str):
    """
    Loads an image from `path` and returns it as BGR numpy array. `path`
    should point to an image file, either a local filename or an url.
    """
    if path.startswith("http"):
        # Set User-Agent to Mozilla because some websites block
        # requests with User-Agent Python
        request = urllib.request.Request(
            path, headers={"User-Agent": "Mozilla/5.0"}
        )
        response = urllib.request.urlopen(request)
        array = np.asarray(bytearray(response.read()), dtype="uint8")
        image = cv2.imdecode(array, -1)  # Loads the image as BGR
    else:
        image = cv2.imread(path)
    return image

In [ ]:
def convert_result_to_image(result, colormap="viridis"):
    """
    Convert network result of floating point numbers to an RGB image with
    integer values from 0-255 by applying a colormap.

    `result` is expected to be a single network result in 1,H,W shape
    `colormap` is a matplotlib colormap.
    See https://matplotlib.org/stable/tutorials/colors/colormaps.html
    """
    cmap = matplotlib.cm.get_cmap(colormap)
    result = result.squeeze(0)
    result = normalize_minmax(result)
    result = cmap(result)[:, :, :3] * 255
    result = result.astype(np.uint8)
    return result

## Load model and get model information

Load the model in Inference Engine with `ie.read_network` and load it to the specified device with `ie.load_network`

In [ ]:
ie = IECore()
net = ie.read_network(
    str(model_xml_path), str(model_xml_path.with_suffix(".bin"))
)
exec_net = ie.load_network(network=net, device_name=DEVICE)

input_key = list(exec_net.input_info)[0]
output_key = list(exec_net.outputs.keys())[0]

network_input_shape = exec_net.input_info[input_key].tensor_desc.dims
network_image_height, network_image_width = network_input_shape[2:]

## Monodepth on Image

### Load, resize and reshape input image

The input image is read with OpenCV, resized to network input size, and reshaped to (N,C,H,W) (H=height, W=width, C=number of channels, N=number of images). 

In [ ]:
IMAGE_FILE = "coco_bike.jpg"
image = load_image(IMAGE_FILE)
# resize to input shape for network
resized_image = cv2.resize(image, (network_image_height, network_image_width))
# reshape image to network input shape NCHW
input_image = np.expand_dims(np.transpose(resized_image, (2, 0, 1)), 0)

### Do inference on image

Do the inference, convert the result to an image, and resize it to the original image shape

In [ ]:
result = exec_net.infer(inputs={input_key: input_image})[output_key]
# convert network result of disparity map to an image that shows
# distance as colors
result_image = convert_result_to_image(result)
# resize back to original image shape. cv2.resize expects shape
# in (width, height), [::-1] reverses the (height, width) shape to match this.
result_image = cv2.resize(result_image, image.shape[:2][::-1])

### Display monodepth image

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 15))
ax[0].imshow(image[:, :, (2, 1, 0)])
ax[1].imshow(result_image);

## Monodepth on Video

By default, only the first 100 frames are processed, in order to quickly check that everything works. Change NUM_FRAMES in the cell below to modify this. Set NUM_FRAMES to 0 to process the whole video.

### Download and load video

In [ ]:
# Video source: https://www.youtube.com/watch?v=fu1xcQdJRws (Public Domain)
VIDEO_FILE = "videos/Coco Walking in Berkeley.mp4"
# Number of video frames to process. Set to 0 to process all frames.
NUM_FRAMES = 100
# Create Path objects for the input video and the resulting video
video_path = Path(VIDEO_FILE)
result_video_path = video_path.with_name(f"{video_path.stem}_monodepth.mp4")

In [ ]:
cap = cv2.VideoCapture(str(video_path))
ret, image = cap.read()
if not ret:
    raise ValueError(f"The video at {video_path} cannot be read.")
FPS = cap.get(cv2.CAP_PROP_FPS)
FRAME_HEIGHT, FRAME_WIDTH = image.shape[:2]
# The format to use for video encoding. VP90 is slow,
# but it works on most systems.
# Try the THEO encoding if you have FFMPEG installed.
FOURCC = cv2.VideoWriter_fourcc(*"VP90")
# FOURCC = cv2.VideoWriter_fourcc(*"THEO")

cap.release()
print(
    f"The input video has a frame width of {FRAME_WIDTH}, "
    f"frame height of {FRAME_HEIGHT} and runs at {FPS} fps"
)

### Do Inference on video and create monodepth video

In [ ]:
frame_nr = 1
start_time = time.perf_counter()
total_inference_duration = 0

cap = cv2.VideoCapture(str(video_path))
out_video = cv2.VideoWriter(
    str(result_video_path),
    FOURCC,
    FPS,
    (FRAME_WIDTH * 2, FRAME_HEIGHT),
)

total_frames = (
    cap.get(cv2.CAP_PROP_FRAME_COUNT) if NUM_FRAMES == 0 else NUM_FRAMES
)
progress_bar = ProgressBar(total=total_frames)
progress_bar.display()

try:
    while cap.isOpened():
        ret, image = cap.read()
        if not ret:
            cap.release()
            break

        if frame_nr == total_frames:
            break

        # Prepare frame for inference
        # resize to input shape for network
        resized_image = cv2.resize(
            image, (network_image_height, network_image_width)
        )
        # reshape image to network input shape NCHW
        input_image = np.expand_dims(np.transpose(resized_image, (2, 0, 1)), 0)

        # Do inference
        inference_start_time = time.perf_counter()
        result = exec_net.infer(inputs={input_key: input_image})[output_key]
        inference_stop_time = time.perf_counter()
        inference_duration = inference_stop_time - inference_start_time
        total_inference_duration += inference_duration

        if frame_nr % 10 == 0:
            clear_output(wait=True)
            progress_bar.display()
            display(
                Pretty(
                    f"Processed frame {frame_nr}. "
                    f"Inference time: {inference_duration:.2f} seconds "
                    f"({1/inference_duration:.2f} FPS)"
                )
            )

        # Transform network result to RGB image
        result_frame = convert_result_to_image(result)[:, :, (2, 1, 0)]
        # Resize to original image shape
        result_frame = cv2.resize(result_frame, (FRAME_WIDTH, FRAME_HEIGHT))
        # Put image and result side by side
        stacked_frame = np.hstack((image, result_frame))
        # Save frame to video
        out_video.write(stacked_frame)

        frame_nr = frame_nr + 1
        progress_bar.progress = frame_nr
        progress_bar.update()

except KeyboardInterrupt:
    print("Processing interrupted.")
finally:
    out_video.release()
    cap.release()
    end_time = time.perf_counter()
    duration = end_time - start_time
    clear_output()
    print(f"Monodepth Video saved to '{str(result_video_path)}'.")
    print(
        f"Processed {frame_nr} frames in {duration:.2f} seconds. "
        f"Total FPS (including video processing): {frame_nr/duration:.2f}."
        f"Inference FPS: {frame_nr/total_inference_duration:.2f} "
    )

### Display monodepth video

In [ ]:
video = Video(result_video_path, width=800, embed=True)
if not result_video_path.exists():
    plt.imshow(stacked_frame)
    raise ValueError(
        "OpenCV was unable to write the video file. Showing one video frame."
    )
else:
    print(f"Showing monodepth video saved at\n{result_video_path.resolve()}")
    print(
        "If you cannot see the video in your browser, please click on the "
        "following link to download the video "
    )
    video_link = FileLink(result_video_path)
    video_link.html_link_str = "<a href='%s' download>%s</a>"
    display(HTML(video_link._repr_html_()))
    display(video)